# Добавление остатков по магазинам

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm import trange, tqdm

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
df = pd.read_csv('csv\hi_tech_5_6_2024.csv')
df

,soup_name,id,date,availability,title,price,old_price,action_sale,sale,category,subcategory,group,count_group,count_group_presence,dif_price,brend
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-06-05,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,0,NaN,televizory-i-elektronika,batareyki-i-zaryadnye-ustroystva,batareyki-i-zaryadnye-ustroystva,76,76,199.750000,energenie
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-06-05,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,0,NaN,televizory-i-elektronika,batareyki-i-zaryadnye-ustroystva,batareyki-i-zaryadnye-ustroystva,76,76,199.750000,energenie
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-06-05,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,0,NaN,televizory-i-elektronika,batareyki-i-zaryadnye-ustroystva,batareyki-i-zaryadnye-ustroystva,76,76,199.750000,energenie
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-06-05,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,0,NaN,televizory-i-elektronika,batareyki-i-zaryadnye-ustroystva,batareyki-i-zaryadnye-ustroystva,76,76,199.750000,energenie
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-06-05,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,0,NaN,televizory-i-elektronika,batareyki-i-zaryadnye-ustroystva,batareyki-i-zaryadnye-ustroystva,76,76,199.750000,energenie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,https://hi-tech.md/kompyuternaya-tehnika/tovary-apple/imac/monoblok-apple-imac-24-2023-blue-mqrr3ru-a-ru/,Т-000089380,2024-06-05,Предзаказ,"Моноблок Apple iMac 24"" 2023 Blue (MQRR3RU/A)",34799.0,38799.0,1,10.31,kompyuternaya-tehnika,tovary-apple,imac,2,0,NaN,apple
5971,https://hi-tech.md/kompyuternaya-tehnika/tovary-apple/macbook/noutbuk-apple-macbook-pro-14.2-2023-silver-mr7j3ru-a-ru/,Т-000089357,2024-06-05,Предзаказ,"Ноутбук Apple MacBook PRO 14.2"" 2023 Silver (MR7J3RU/A)",35399.0,NaN,0,NaN,kompyuternaya-tehnika,tovary-apple,macbook,15,7,1.479874,apple
5972,https://hi-tech.md/kompyuternaya-tehnika/tovary-apple/macbook/noutbuk-apple-macbook-pro-14.2-2023-silver-mr7k3ru-a-ru/,Т-000089359,2024-06-05,Предзаказ,"Ноутбук Apple MacBook PRO 14.2"" 2023 Silver (MR7K3RU/A)",39399.0,NaN,0,NaN,kompyuternaya-tehnika,tovary-apple,macbook,15,7,1.479874,apple
5973,https://hi-tech.md/kompyuternaya-tehnika/tovary-apple/macbook/noutbuk-apple-macbook-pro-14.2-2023-space-grey-mtl83ru-a/,Т-000089360,2024-06-05,Предзаказ,"Ноутбук Apple MacBook PRO 14.2"" 2023 Space Grey (MTL83RU/A)",39399.0,NaN,0,NaN,kompyuternaya-tehnika,tovary-apple,macbook,15,7,1.479874,apple


In [4]:
def count_sity(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    data = []
    data.append((soup.findAll('div', class_='ty-product-block__sku'))[0].text.split()[1])
    element = soup.findAll('div', class_='ty-warehouses-store__availability')
    for i in (element):
        data.append(i.text.strip()
                    .replace('В наличии', '1')
                    .replace('Нет в наличии','0')
                    .replace('Под заказ (1-3 дня)','0')
                    .replace('nan','0'))
    return data

In [5]:
df['count_sity'] = df['soup_name'].apply(count_sity)

In [6]:
t = pd.DataFrame({ 'id': df['count_sity'].str[0] ,
                  'Тир_Юности_14А': df['count_sity'].str[1], 
                  'Ц_Тир_25_Октября_69': df['count_sity'].str[2],
                  'Ц_Бенд_Спортивная_5': df['count_sity'].str[3],
                  'Бенд_Советская_5': df['count_sity'].str[4],
                  'Бенд_Советская_8': df['count_sity'].str[5], 
                  'Слоб_Ленина_97А': df['count_sity'].str[6],
                  'Днестр_Строителей_18': df['count_sity'].str[7],
                  'Первом_Ленина_88': df['count_sity'].str[8],
                  'Григ_КМаркса_181А': df['count_sity'].str[9], 
                  'Григ_КМаркса_172': df['count_sity'].str[10],
                  'Дуб_Горького_29': df['count_sity'].str[11], 
                  'Рыбн_Вальченко_43/1': df['count_sity'].str[12], 
                  'Рыбн_Кирова_135': df['count_sity'].str[13],
                  'Кам_Кирова_240': df['count_sity'].str[14]})
t

,id,Тир_Юности_14А,Ц_Тир_25_Октября_69,Ц_Бенд_Спортивная_5,Бенд_Советская_5,Бенд_Советская_8,Слоб_Ленина_97А,Днестр_Строителей_18,Первом_Ленина_88,Григ_КМаркса_181А,Григ_КМаркса_172,Дуб_Горького_29,Рыбн_Вальченко_43/1,Рыбн_Кирова_135,Кам_Кирова_240
0,Т-000088228,1,1,1,1,0,1,1,1,1,1,1,1,1,1
1,Т-000088229,1,1,1,1,0,1,1,1,1,1,1,1,1,1
2,Т-000088221,1,1,1,1,1,1,1,1,1,1,1,0,1,1
3,Т-000088222,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,Т-000088230,1,1,1,1,0,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,Т-000089380,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии
5971,Т-000089357,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии
5972,Т-000089359,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии
5973,Т-000089360,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии,Нет в наличии


In [7]:
df = df.merge(t, on='id')

In [8]:
df.drop(['count_sity'], axis= 1 , inplace= True )

In [9]:
df.head(1)

,soup_name,id,date,availability,title,price,old_price,action_sale,sale,category,subcategory,group,count_group,count_group_presence,dif_price,brend,Тир_Юности_14А,Ц_Тир_25_Октября_69,Ц_Бенд_Спортивная_5,Бенд_Советская_5,Бенд_Советская_8,Слоб_Ленина_97А,Днестр_Строителей_18,Первом_Ленина_88,Григ_КМаркса_181А,Григ_КМаркса_172,Дуб_Горького_29,Рыбн_Вальченко_43/1,Рыбн_Кирова_135,Кам_Кирова_240
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-06-05,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,0,NaN,televizory-i-elektronika,batareyki-i-zaryadnye-ustroystva,batareyki-i-zaryadnye-ustroystva,76,76,199.75,energenie,1,1,1,1,0,1,1,1,1,1,1,1,1,1


In [11]:
df['Ц_Бенд_Спортивная_5'].unique()

array(['1', '0', 'Нет в наличии', nan], dtype=object)

In [12]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_sity_'+str(today.day)+'_'+str(today.month)+'_'+str(today.year)+'.csv', index=False)  